In [1]:
from coral.dataprovider import DataProvider
from coral import toolx
from Utils import *

In [2]:
import os
import sys

sys.path.append('/Users/desho/echusOverlook')
from eo.inventory import Inventory
from eo.rma import RMA, MissionSegment
from eo.crew import Crew
from eo.mission_needs import *

# Initialize CORAL

In [3]:
toolx.init_system()

<pyArango.connection.Connection object at 0x7fa5f9e17d90> eO
Init system collections
Truncate system collection: SYS_ID
Init system collection: SYS_ID
Can not create collection: SYS_ID
Truncate system collection: SYS_ObjectTypeID
Init system collection: SYS_ObjectTypeID
Can not create collection: SYS_ObjectTypeID
Init system collection: SYS_UserProfile
Can not create collection: SYS_UserProfile
Deleting old data
<pyArango.connection.Connection object at 0x7fa5f9e41430> eO
Removing process edges
Truncate system collection: SYS_ProcessInput
Truncate system collection: SYS_ProcessOutput
Removing SYS_Process 
<pyArango.connection.Connection object at 0x7fa5f9e81c10> eO
Removing DDT_Brick 
<pyArango.connection.Connection object at 0x7fa5f9e91700> eO
Removing SDT_CUBES 
Removing SDT_Campaign 
Removing SDT_Techno-economics 
Removing SDT_Simulation 
Removing SDT_Inventory 
Removing SDT_RMA 
Removing SDT_Crew 
Uploading new data
<pyArango.connection.Connection object at 0x7fa5f9e17a90> eO
Ontol

In [4]:
from coral.arango_service import ArangoService
from pyArango.connection import *

In [5]:
__arango_conn = Connection(arangoURL='http://127.0.0.1:8529',username='root', password='echusOverlook')
arango_service = ArangoService(__arango_conn, 'eO')

# Uploading to ArangoDB

In [6]:
import pandas as pd
import os

In [7]:
cs1_dir = '/Users/desho/CORAL-eO/example/eO/data/case_study_1'
for file in os.listdir(cs1_dir):
    if file[-4:] == 'xlsx':
        name = cs1_dir + '/' + file
        df = pd.read_excel(name)
        if 'Process' in df.columns:
            df['Process'] = df['Process'].ffill()
        df = df.fillna(0)
        df.to_csv(name[:-5] + '.tsv', sep='\t', index=False)
        print('created', name[:-5] + '.tsv')

created /Users/desho/CORAL-eO/example/eO/data/case_study_1/crew.tsv
created /Users/desho/CORAL-eO/example/eO/data/case_study_1/inventory_tr2.tsv
created /Users/desho/CORAL-eO/example/eO/data/case_study_1/inventory_pd.tsv
created /Users/desho/CORAL-eO/example/eO/data/case_study_1/RMA.tsv
created /Users/desho/CORAL-eO/example/eO/data/case_study_1/inventory_tr1.tsv
created /Users/desho/CORAL-eO/example/eO/data/case_study_1/inventory_sf.tsv


In [8]:
toolx.update_core('case_study_1/RMA.tsv', 'RMA')

type_name RMA
Doing RMA: case_study_1/RMA.tsv
<pyArango.connection.Connection object at 0x7fa5fa06a490> eO
size=4


In [9]:
toolx.update_core('case_study_1/inventory_pd.tsv', 'Inventory')

type_name Inventory
Doing Inventory: case_study_1/inventory_pd.tsv
<pyArango.connection.Connection object at 0x7fa5fa11e850> eO
size=2


In [10]:
toolx.update_core('case_study_1/inventory_sf.tsv', 'Inventory')

type_name Inventory
Doing Inventory: case_study_1/inventory_sf.tsv
<pyArango.connection.Connection object at 0x7fa5fa1d5e50> eO
Can not create a collection for type Inventory
size=6


In [11]:
toolx.update_core('case_study_1/inventory_tr1.tsv', 'Inventory')

type_name Inventory
Doing Inventory: case_study_1/inventory_tr1.tsv
<pyArango.connection.Connection object at 0x7fa5fa3aad90> eO
Can not create a collection for type Inventory
size=6


In [12]:
toolx.update_core('case_study_1/inventory_tr2.tsv', 'Inventory')

type_name Inventory
Doing Inventory: case_study_1/inventory_tr2.tsv
<pyArango.connection.Connection object at 0x7fa5fa3aa220> eO
Can not create a collection for type Inventory
size=6


In [13]:
toolx.update_core('case_study_1/crew.tsv', 'Crew')

type_name Crew
Doing Crew: case_study_1/crew.tsv
<pyArango.connection.Connection object at 0x7fa5fa3aac70> eO
size=4


# Database queries

#### Create RMA object

In [14]:
aql = 'FOR item in SDT_RMA RETURN item'
data = __arango_conn['eO'].AQLQuery(aql)

RMA_CS1 = RMA(data)

pd | 210 | earth, earth-orbit, mars-orbit, mars
tr1 | 410 | earth, earth-orbit, mars-orbit, mars
tr2 | 410 | mars, mars-orbit, earth-orbit, earth
sf | 500 | mars


#### Create inventories

In [15]:
aql = 'FOR item in SDT_Inventory FILTER CONTAINS(item.Name, "Inventory-pd") == true RETURN item'
data = __arango_conn['eO'].AQLQuery(aql)
inv_pd = Inventory('root1', data=data)

parent BioM child Crop reactor/Plant growth chamber
parent BioM child Crop storage


In [ ]:
aql = 'FOR item in SDT_Inventory FILTER CONTAINS(item.Name, "Inventory-tr1") == true RETURN item'
data = __arango_conn['eO'].AQLQuery(aql)
inv_tr1 = Inventory('root2', data=data)

In [ ]:
aql = 'FOR item in SDT_Inventory FILTER CONTAINS(item.Name, "Inventory-tr2") == true RETURN item'
data = __arango_conn['eO'].AQLQuery(aql)
inv_tr2 = Inventory('root3', data=data)

In [ ]:
aql = 'FOR item in SDT_Inventory FILTER CONTAINS(item.Name, "Inventory-sf") == true RETURN item'
data = __arango_conn['eO'].AQLQuery(aql)
inv_sf = Inventory('root4', data=data)

#### Add inventories to RMA segments

In [ ]:
RMA_CS1.segments

In [ ]:
inventories = {'pd':inv_pd, 'tr1':inv_tr1, 'tr2':inv_tr2, 'sf':inv_sf}

for key in RMA_CS1.segments.keys():
    RMA_CS1.segments[key].set_inventory(inventories[key])

#### Create crew object

In [ ]:
aql = 'FOR item in SDT_Crew RETURN item'
data = __arango_conn['eO'].AQLQuery(aql)
CREW_CS1 = Crew.from_arango(data)

#### Add crew to RMA object

In [ ]:
for segment in RMA_CS1.segments.values():
    if segment.crew_count == 4:
        segment.set_crew(CREW_CS1)

# Calculations

#### Mission needs

In [ ]:
calc_mission_needs(RMA_CS1)

In [ ]:
import matplotlib.pyplot as plt

needs = {'tr1':{'kcal': 17695.094169059117, 'sensible_heat': 9469.354860009202, 'latent_heat': 8038.609862946554, 'water_vapor': 3047.489821641586, 'sweat': 208.04796235497815, 'O2_consumption': 1221.9440446763515, 'CO2_output': 1553.7148016234069}, \
        'tr2': {'kcal': 17695.094169059117, 'sensible_heat': 9469.354860009202, 'latent_heat': 8038.609862946554, 'water_vapor': 3047.489821641586, 'sweat': 208.04796235497815, 'O2_consumption': 1221.9440446763515, 'CO2_output': 1553.7148016234069}, \
        'sf': {'kcal': 21579.383132998926, 'sensible_heat': 11547.993731718538, 'latent_heat': 9803.18275969092, 'water_vapor': 3716.451002001934, 'sweat': 253.7170272621685, 'O2_consumption': 1490.1756642394535, 'CO2_output': 1894.774148321228}}

df = pd.DataFrame.from_dict(needs)

In [ ]:
df.plot(y=["tr1", "tr2", "sf"], kind="bar")
plt.show()

#### ESM